In [ ]:
#Import the libraries
import pandas as pd
import numpy as np
import scipy.stats as sp
import matplotlib.pyplot as plt
from sklearn import metrics
import math
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

In [ ]:
#Preprocessing

###A function to split a string based on seprator and position of separator
def split(strng, sep, pos):
    strng = strng.split(sep)
    return sep.join(strng[:pos]), sep.join(strng[pos:])

###Merge submission and visit data
df_sub =pd.read_csv('E:/R workspace/recruit-restaurant-visitor-forecasting/sample_submission.csv')
df_sub[['air_store_id','visit_date']] = df_sub.id.apply( 
   lambda x: pd.Series(split(str(x), "_", 2))) 
df_sub = df_sub.drop(columns=['id'])
df = pd.read_csv('E:/R workspace/recruit-restaurant-visitor-forecasting/air_visit_data.csv')
medianValue = round(df['visitors'].median())
df = df.append(df_sub, ignore_index=True, sort=False)

###Resample dates and perform median imputation
df.index = pd.to_datetime(df['visit_date'])
df = df.groupby('air_store_id').apply(lambda g: g['visitors'].resample('D').sum()).reset_index()
df['visit_date'] = df['visit_date'].dt.strftime('%Y-%m-%d')
subm = df[df['visit_date'] >= '2017-04-23'].copy()
mask = (df['visitors'] == 0) & (df['visit_date'] < '2017-04-23')
df['visitors'][mask] = medianValue

###Remove unnecessary restaurant data
lst = subm.air_store_id.unique()
subDF = df[df['air_store_id'].isin(lst)]
subDF['visitors_org'] = subDF['visitors']
subDF['visitors'] = np.log1p(subDF['visitors'])

In [ ]:
#MA model
i=0
dfObjSubm = pd.DataFrame(columns=['id', 'visitors'])
for item in lst:
    i+=1
    print(i)
    print(item)
    dr = subDF[subDF['air_store_id'] == item] 
    train_df = dr.loc[dr.visit_date < '2017-04-23', ~dr.columns.isin(['air_store_id','visitors_org'])]
    train_df.set_index('visit_date', inplace=True)
    test_df = dr.loc[dr.visit_date >= '2017-04-23', ~dr.columns.isin(['visitors_org'])]
    model = pm.auto_arima(train_df.visitors, start_p=0, start_q=1,
                      test='adf',       
                      max_p=0, max_q=5, 
                      m=1,              
                      d=0,          
                      seasonal=False,   
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
    n_periods = 39
    fc = model.predict(n_periods=n_periods)
    test_df['id'] = test_df.apply(lambda row: row.air_store_id +'_'+ str(row.visit_date).split()[0], axis=1)
    test_df['visitors'] = fc
    dfObjSubm = pd.concat([dfObjSubm, test_df[['id','visitors']]])
dfObjSubm['visitors'] = round(np.expm1(dfObjSubm['visitors']))
dfObjSubm = dfObjSubm.reset_index(drop=True)
dfObjSubm.to_csv('file_MA.csv', index=False)